## 1. Посмотреть документацию к API GitHub, разобраться как вывести список репозиториев для конкретного пользователя, сохранить JSON-вывод в файле *.json.

In [90]:
import json
import numpy as np
import requests
from pprint import pprint

In [91]:
response = requests.get('https://api.github.com/users/AlekseyKozubenko/repos')
# pprint(response.json())
names = response.json()
for name in names:
    print(name['name'])

# в файл сохраняю полный набор данных, полученные по запросу
with open('data.json', 'w') as f:
    f.write(json.dumps(response.text))

Higher_Mathematics
manchenkoff
Parsing
project
Python_for_Data_Science
python_gb
repo-github
repo-gui
webpack-bootstrap


## 2. Работа будет состоять с недокументированным API. Нужно ввести релевантный запрос на сайте https://www.delivery-club.ru/search
(а) из предложенных точек с помощью API найти долю (в %) с бесплатной и платной доставкой. Для каждой категории рассчитать среднюю минимальную стоимость заказа.

(б) для каждой из категорий из пункта (а) рассчитать долю (в %) магазинов и ресторанов

In [92]:
# proxies = {
#     'https': '91.197.77.118:443',
# }

# headers = {
#     'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36',
# }

params = {
    'latitude': '55.795808',
    'longitude': '49.224417',
    'query': 'манты',
    'cacheBreaker' : '1662195263'
}

url = 'https://api.delivery-club.ru/api1.2/vendors/search'

response = requests.get(url=url, params=params)

print(response.status_code)


200


In [96]:
vendors = response.json()['vendors']
# for vendor in vendors:
  # print(vendor['name'], vendor['delivery']['price']['value'])

count_total = 0
count_free = 0
order_price_free = []
order_price_not_free = []
shops_free = 0
resto_free = 0
shops_not_free = 0
resto_not_free = 0



for vendor in vendors:
  count_total += 1
  if int(vendor['delivery']['price']['value']) == 0:
    count_free += 1
    if int(vendor['categoryId']) == 1:
      resto_free += 1
    else:
      shops_free += 1
    order_price_free.append(int(vendor['delivery']['minOrderPrice']['value']))
  else:
    if int(vendor['categoryId']) == 1:
      resto_not_free += 1
    else:
      shops_not_free += 1
    order_price_not_free.append(int(vendor['delivery']['minOrderPrice']['value']))

# print(order_price_free)
# print(order_price_not_free)

part_free = str(round(count_free*100/count_total, 2))+'%'
part_not_free = str(round((count_total-count_free)*100/count_total, 2))+'%'
# print(part_free)
part_resto_free = str(round(resto_free*100/count_free, 2))+'%'
part_shops_free = str(round(shops_free*100/count_free, 2))+'%'

part_resto_not_free = str(round(resto_not_free*100/(count_total-count_free), 2))+'%'
part_shops_not_free = str(round(shops_not_free*100/(count_total-count_free), 2))+'%'

# print(part_shops_free)

result = {'Бесплатная доставка': 
          {'Доля продавцов': part_free, 
           'Средняя стоимость заказа': np.around(np.mean(order_price_free),2), 
           'Минимальная стоимость заказа': np.amin(order_price_free),
           'Тип продавца':
              {'Магазины': part_shops_free, 
              'Рестораны': part_resto_free}}, 
          'Платная доставка': 
          {'Доля продавцов': part_not_free, 
           'Средняя стоимость заказа': np.around(np.mean(order_price_not_free),2), 
           'Минимальная стоимость заказа': np.amin(order_price_not_free),
           'Тип продавца': 
               {'Магазины': part_shops_not_free, 
                'Рестораны': part_resto_not_free}}
}
pprint(result)

{'Бесплатная доставка': {'Доля продавцов': '57.69%',
                         'Минимальная стоимость заказа': 290,
                         'Средняя стоимость заказа': 982.67,
                         'Тип продавца': {'Магазины': '20.0%',
                                          'Рестораны': '80.0%'}},
 'Платная доставка': {'Доля продавцов': '42.31%',
                      'Минимальная стоимость заказа': 1,
                      'Средняя стоимость заказа': 91.82,
                      'Тип продавца': {'Магазины': '18.18%',
                                       'Рестораны': '81.82%'}}}


In [94]:
# for vendor in vendors:
#   print(vendor['name'], vendor['delivery']['minOrderPrice']['value'], vendor['delivery']['price']['value'], vendor['categoryId']) 

In [95]:
# pprint(response.json())